In [1]:
import os 
import numpy as np 
import pandas as pd
from collections import defaultdict
import itertools
import ast

In [5]:
datadir = '../../datasets/the-movies-dataset/'

In [6]:
small_ratings_file = datadir + 'ratings_small.csv'
df = pd.read_csv(small_ratings_file, delimiter=',', usecols=[0,1,2])

In [7]:
df.dataframeName = 'ratings_small.csv'
print(df.columns)

Index([u'userId', u'movieId', u'rating'], dtype='object')


In [8]:
# df = df.drop(columns=['timestamp'])
print(df.head(5))

   userId  movieId  rating
0       1       31     2.5
1       1     1029     3.0
2       1     1061     3.0
3       1     1129     2.0
4       1     1172     4.0


In [9]:
print(max(df['movieId']))
print(min(df['movieId']))
print(type(df['movieId'][0]))

163949
1
<type 'numpy.int64'>


In [10]:
print (max(df['userId']))
print (min(df['userId']))
print(type(df['userId'][0]))

671
1
<type 'numpy.int64'>


In [11]:
# userid_set = set()
# movieid_set = set()

# for i in df['userId']:
#     if i not in userid_set:
#         userid_set.add(i)

# for i in df['movieId']:
#     if i not in movieid_set:
#         movieid_set.add(i)

In [9]:
# print(len(userid_set))

671


In [12]:
# print(len(movieid_set))

NameError: name 'movieid_set' is not defined

In [11]:
# TODO
# Cast all floats to int since comparison and access
# can be problematic later on in dict keys

In [12]:
# creating a hash map of  movieId -> list of userIds 
# First create a threshold for the ratings to consider: rating_threshold
# Then prune the hashmap to only include the movies which have been rated 
# (clearing the ratings_threshold) by at least say 50 number of users (usernum_threshold)

# Could probably use faster pandas dataframe lambda methods to chunk out
# lots of rows which have a rating value less than the threshold


In [16]:
rating_threshold = 1.5
movie_user_dict = defaultdict(list)
df = df[df['rating'] >= rating_threshold]
df.head()


,userId,movieId,rating
4,1,1172,4.0
8,1,1339,3.5
12,1,1953,4.0
13,1,2105,4.0
20,2,10,4.0


In [17]:

# This step can be done efficiently
# with lambda apply on pandas df
for index, row in df.iterrows():    
#     movId = row['movieId']
    movId = int(row['movieId'])
#     usrId = row['userId']
    usrId = int(row['userId'])
    rating = row['rating']       
    
    if rating >= rating_threshold:
        movie_user_dict[movId].append(usrId)

In [18]:
usernum_threshold = 50
final_dict = {}

for key,val in movie_user_dict.items():
    if len(val) >= usernum_threshold:
        final_dict[key] = val


In [19]:
len(final_dict)

258

In [20]:
# Calculate the total number of users. 
# Only consider the movies in final_dict
# i.e after all the final thresholding

total_users = set()
for mov in final_dict.keys():
    row = set(final_dict[mov])
    total_users = total_users.union(row)

num_users = len(total_users)
print(num_users)
    

661


In [21]:
def findnum_common_elements(list1, list2):
    return len(set(list1).intersection(list2))

In [41]:
# movie_idlist = list(final_dict.keys())
# # print(movie_idlist)

[1, 2, 2987, 1028, 6, 10, 11, 1036, 33794, 54286, 527, 16, 17, 32587, 21, 1625, 25, 539, 541, 32, 2081, 34, 36, 1573, 39, 3114, 555, 1580, 47, 1584, 1073, 50, 1079, 1080, 68157, 62, 1089, 1090, 1035, 2628, 1097, 586, 587, 588, 589, 590, 592, 593, 595, 597, 953, 1380, 5989, 608, 104, 56367, 7438, 110, 111, 1136, 1653, 49272, 2683, 1148, 2174, 2916, 4226, 5995, 2692, 648, 3948, 2700, 141, 3949, 2599, 1682, 150, 1732, 40815, 2716, 44191, 161, 165, 3751, 1704, 1193, 3671, 1196, 1197, 1198, 1199, 1200, 1203, 1206, 1207, 1208, 1721, 1210, 8360, 6333, 1213, 1214, 58559, 7361, 1219, 1220, 1221, 1222, 1225, 2762, 1230, 3793, 1234, 1240, 6874, 2268, 733, 1246, 223, 736, 1250, 1252, 5349, 231, 745, 1258, 551, 2797, 750, 1265, 2291, 2804, 1270, 1784, 68954, 253, 1278, 8961, 260, 8874, 1288, 778, 1291, 780, 4878, 1923, 2324, 4886, 1304, 2329, 1307, 79132, 4896, 292, 293, 296, 2012, 2858, 300, 2791, 2355, 3471, 3897, 1247, 316, 318, 5952, 4011, 5445, 6711, 5418, 1356, 1358, 3408, 337, 339, 344, 858,

In [22]:
count_matrix = {}

# for pair in itertools.combinations(movie_idlist, r=2):
for pair in itertools.combinations(final_dict.keys(), r=2):
    mov1, mov2 = pair
    count_matrix[pair] = findnum_common_elements(final_dict[mov1], final_dict[mov2])

In [23]:
print(len(count_matrix))

33153


In [24]:
# Now just add the marginal counts to the count_matrix dict

for k in final_dict.keys():
    pair = (k,k)
    count_matrix[pair] = len(final_dict[k])

In [25]:
count_matrix[(10,11)]

12

In [26]:
count_matrix[(10.0,10.0)]

65

In [27]:
maxval = max([v for k,v in count_matrix.items()])
print(maxval)    

286


In [27]:
def marginal_prob(movie_id, count_matrix, num_users):
    '''function to get the marginal prob:
        P(movie_id1)       
    '''
    margn_count = count_matrix[(movie_id, movie_id)]
    return margn_count/num_users

def joint_prob(movie_id1, movie_id2, count_matrix, num_users):
    '''function to get the joint prob:
        P(movie_id1, movie_id2)       
    '''
    if (movie_id1, movie_id2) not in count_matrix.keys():
        key = (movie_id2, movie_id1)
    else:
        key = (movie_id1, movie_id2)
    
    joint_count = count_matrix[key]    
    return joint_count/num_users


def conditional_prob(movie_id1, movie_id2, count_matrix):
    '''function to get the conditional prob:
        P(movie_id1 | movie_id2)       
    '''
    if (movie_id1, movie_id2) not in count_matrix.keys():
        key = (movie_id2, movie_id1)
    else:
        key = (movie_id1, movie_id2)
    
    joint_count = count_matrix[key]
    margn_count = count_matrix[movie_id2, movie_id2]
    
    return joint_count/margn_count

In [28]:
count_matrix.keys()

dict_keys([(10, 17), (10, 39), (10, 47), (10, 50), (10, 110), (10, 150), (10, 253), (10, 296), (10, 349), (10, 454), (10, 480), (10, 500), (10, 508), (10, 509), (10, 527), (10, 551), (10, 589), (10, 590), (10, 592), (10, 318), (10, 356), (10, 736), (10, 778), (10, 1197), (10, 1580), (10, 1721), (10, 2028), (10, 2762), (10, 2858), (10, 2959), (10, 3949), (10, 44191), (10, 34), (10, 141), (10, 260), (10, 357), (10, 364), (10, 367), (10, 440), (10, 541), (10, 588), (10, 858), (10, 903), (10, 910), (10, 913), (10, 919), (10, 1028), (10, 1036), (10, 1073), (10, 1079), (10, 1089), (10, 1097), (10, 1136), (10, 1196), (10, 1198), (10, 1200), (10, 1206), (10, 1208), (10, 1210), (10, 1213), (10, 1214), (10, 1219), (10, 1220), (10, 1222), (10, 1225), (10, 1230), (10, 1240), (10, 1258), (10, 1259), (10, 1265), (10, 1270), (10, 1278), (10, 1288), (10, 1291), (10, 1307), (10, 1356), (10, 1374), (10, 1380), (10, 1387), (10, 1961), (10, 1968), (10, 2000), (10, 2081), (10, 2115), (10, 2174), (10, 2194)

In [29]:
N = num_users
marginal_prob(10, count_matrix, N)

0.09833585476550681

In [30]:
joint_prob(10, 11, count_matrix, N)

0.018154311649016642

In [31]:
conditional_prob(10, 8961, count_matrix)

0.1782178217821782

In [32]:
# Sample example
# Conditioning CAN be less than the marginal! 
# ref: 2018-box-paper table-1

for k in final_dict.keys():
    p1 = conditional_prob(10, k, count_matrix)
    p2 = marginal_prob(10, count_matrix, N)
    if p1 <= p2:
        print(k, p1)

318 0.0944055944055944
903 0.06451612903225806
913 0.03571428571428571
1225 0.06976744186046512
1230 0.047619047619047616
1247 0.04
5995 0.09259259259259259
111 0.07692307692307693
745 0.07547169811320754
924 0.08695652173913043
36 0.08860759493670886
608 0.08205128205128205
1358 0.058823529411764705
923 0.08333333333333333
79132 0.08602150537634409
58559 0.0673076923076923
750 0.05555555555555555
899 0.057692307692307696
904 0.09411764705882353
908 0.06493506493506493
912 0.09278350515463918
920 0.08928571428571429
1193 0.08064516129032258
1203 0.046875
1252 0.04411764705882353
1394 0.07407407407407407
2395 0.05555555555555555
50872 0.058823529411764705
59315 0.09230769230769231
60069 0.08928571428571429
25 0.08695652173913043
1304 0.09230769230769231
2599 0.07936507936507936
68954 0.07547169811320754
46578 0.09375
56367 0.0847457627118644


In [32]:
# METADATA ACCESS -- for genres.

metadata_file = datadir + 'movies_metadata.csv'
df_meta = pd.read_csv(metadata_file, delimiter=',')


In [33]:
df_meta.columns

Index([u'adult', u'belongs_to_collection', u'budget', u'genres', u'homepage',
       u'id', u'imdb_id', u'original_language', u'original_title', u'overview',
       u'popularity', u'poster_path', u'production_companies',
       u'production_countries', u'release_date', u'revenue', u'runtime',
       u'spoken_languages', u'status', u'tagline', u'title', u'video',
       u'vote_average', u'vote_count'],
      dtype='object')

In [34]:
df_meta.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [35]:
df_meta = df_meta[['genres', 'id']]

In [36]:
df_meta.head(5)

,genres,id
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357
4,"[{'id': 35, 'name': 'Comedy'}]",11862


In [38]:
print df_meta.loc[0].id
print df_meta.loc[0, 'id']

862
862


In [39]:
print(df_meta.loc[0,'genres'])

[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]


In [90]:
x = df_meta.loc[0,'genres']
print(type(x))
print(x)

<type 'str'>
[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]


In [41]:
s = x.split('},')
print(s)

["[{'id': 16, 'name': 'Animation'", " {'id': 35, 'name': 'Comedy'", " {'id': 10751, 'name': 'Family'}]"]


In [48]:
tmp = ast.literal_eval(x)
print(type(tmp))
print(tmp)

<type 'list'>
[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]


In [66]:
x = 'Science  Fiction'
y = x.replace(" ", "-")
print y


Science--Fiction


In [67]:
ids = []
gen = []

for d in tmp:
    ids.append(d['id'])
    g = d['name']
    g.replace(" ", "-") # change all whitespaces to hyphen
    gen.append(g)

print(ids, gen)

([16, 35, 10751], ['Animation', 'Comedy', 'Family'])


In [47]:
s0=s[0]
print(s0)

NameError: name 's' is not defined

In [43]:
i = s0.find('id')
n = s0.find('name')
print(s0[i+5:n-3]) # the genre id
print(s0[n+8:-1])

NameError: name 's0' is not defined

In [46]:
s1 = s[1]
i = s1.find('id')
n = s1.find('name')
print(s1[i+5:n-3]) # the genre id
print(s1[n+8:-1])

35
Comedy


In [75]:

def get_genres(genres_json, id_to_genre):
    '''
    id_to_genre: its like a global dict to store the mappings 
                from the genre-id to genre-name
    '''
    if not genres_json:
        return None
    
    tmp = ast.literal_eval(genres_json)
    ids_list = []
    gen_list = []           
    
    for d in tmp:
        id_num = d['id']
        genre_name = d['name']
        genre_name = genre_name.replace(" ", "-") # change all whitespaces to hyphen
        ids_list.append(id_num)
        gen_list.append(genre_name)
        
        if id_num not in id_to_genre:
            id_to_genre[id_num] = genre_name
    
    return ids_list, gen_list, id_to_genre


In [98]:
genid_dict = {}
genname_dict = {}
gen_counts = defaultdict(int) # counting the genre-genre pairs
id_to_genre = {}

for index, row in df_meta.iterrows():    
    
    try:
        movId = int(row['id'])
        gen_json = row['genres']
    
        ids, genres, id_to_genre = get_genres(gen_json, id_to_genre)    
        genid_dict[movId] = ids
        genname_dict[movId] = genres

        for i in ids:
            singleton_pair = (i, i)
            gen_counts[singleton_pair] += 1

        for pair in itertools.combinations(ids, r=2):
            id1, id2 = pair
            gen_counts[pair] += 1
            
    except Exception as e:
        print(index, row['id'])
        repr(e)


(19730, '1997-08-20')
(29503, '2012-09-29')
(35587, '2014-01-01')


In [92]:
print(id_to_genre)
print(len(id_to_genre))

{10752: 'War', 80: 'Crime', 10402: 'Music', 35: 'Comedy', 36: 'History', 878: 'Science-Fiction', 37: 'Western', 12: 'Adventure', 10770: 'TV-Movie', 14: 'Fantasy', 16: 'Animation', 10769: 'Foreign', 18: 'Drama', 9648: 'Mystery', 53: 'Thriller', 99: 'Documentary', 27: 'Horror', 28: 'Action', 10749: 'Romance', 10751: 'Family'}
20


In [93]:
genid_dict

{2: [18, 80],
 3: [18, 35],
 5: [80, 35],
 6: [28, 53, 80],
 11: [12, 28, 878],
 12: [16, 10751],
 13: [35, 18, 10749],
 14: [18],
 15: [9648, 18],
 16: [18, 80, 10402],
 17: [27, 53, 9648],
 18: [12, 14, 28, 53, 878],
 19: [18, 878],
 20: [18, 10749],
 21: [99],
 22: [12, 14, 28],
 24: [28, 80],
 25: [18, 10752],
 26: [18],
 27: [18, 10402, 10749],
 28: [18, 10752],
 30: [16, 878],
 262176: [18],
 33: [37],
 35: [16, 35, 10751],
 38: [878, 18, 10749],
 131116: [18],
 393263: [16],
 55: [18, 53],
 43700: [28, 35, 37],
 58: [12, 14, 28],
 59: [18, 53, 80],
 62: [878, 9648, 12],
 63: [878, 53, 9648],
 64: [18, 10749],
 65: [18],
 66: [80, 18, 53],
 67: [53, 80, 18],
 68: [35, 878],
 69: [18, 10402, 10749],
 70: [18],
 71: [18, 35, 10402],
 73: [18],
 74: [12, 53, 878],
 75: [35, 14, 878],
 76: [18, 10749],
 77: [9648, 53],
 78: [878, 18, 53],
 79: [18, 12, 28, 36],
 80: [18, 10749],
 81: [12, 16, 14],
 82: [28, 12, 80, 53],
 83: [18, 53],
 85: [12, 28],
 86: [18, 10749],
 87: [12, 28],
 

In [94]:
genname_dict

{2: ['Drama', 'Crime'],
 3: ['Drama', 'Comedy'],
 5: ['Crime', 'Comedy'],
 6: ['Action', 'Thriller', 'Crime'],
 11: ['Adventure', 'Action', 'Science-Fiction'],
 12: ['Animation', 'Family'],
 13: ['Comedy', 'Drama', 'Romance'],
 14: ['Drama'],
 15: ['Mystery', 'Drama'],
 16: ['Drama', 'Crime', 'Music'],
 17: ['Horror', 'Thriller', 'Mystery'],
 18: ['Adventure', 'Fantasy', 'Action', 'Thriller', 'Science-Fiction'],
 19: ['Drama', 'Science-Fiction'],
 20: ['Drama', 'Romance'],
 21: ['Documentary'],
 22: ['Adventure', 'Fantasy', 'Action'],
 24: ['Action', 'Crime'],
 25: ['Drama', 'War'],
 26: ['Drama'],
 27: ['Drama', 'Music', 'Romance'],
 28: ['Drama', 'War'],
 30: ['Animation', 'Science-Fiction'],
 262176: ['Drama'],
 33: ['Western'],
 35: ['Animation', 'Comedy', 'Family'],
 38: ['Science-Fiction', 'Drama', 'Romance'],
 131116: ['Drama'],
 393263: ['Animation'],
 55: ['Drama', 'Thriller'],
 43700: ['Action', 'Comedy', 'Western'],
 58: ['Adventure', 'Fantasy', 'Action'],
 59: ['Drama', 'Th

In [95]:
gen_counts

defaultdict(int,
            {(12, 14): 551,
             (12, 16): 273,
             (12, 18): 882,
             (12, 27): 130,
             (12, 28): 426,
             (12, 35): 655,
             (12, 36): 143,
             (12, 37): 116,
             (12, 53): 474,
             (12, 80): 198,
             (12, 99): 28,
             (12, 878): 457,
             (12, 9648): 97,
             (12, 10402): 22,
             (12, 10749): 263,
             (12, 10751): 588,
             (12, 10752): 117,
             (12, 10769): 53,
             (12, 10770): 26,
             (14, 12): 131,
             (14, 16): 129,
             (14, 18): 239,
             (14, 27): 281,
             (14, 28): 187,
             (14, 35): 275,
             (14, 36): 8,
             (14, 37): 6,
             (14, 53): 172,
             (14, 80): 13,
             (14, 99): 6,
             (14, 878): 428,
             (14, 9648): 93,
             (14, 10402): 52,
             (14, 10749): 253,
             (1

In [100]:
genname_counts = {}
for k,v in gen_counts.items():
    k1, k2 = k
    g1 = id_to_genre[k1]
    g2 = id_to_genre[k2]
    genname_counts[g1, g2] = v

In [101]:
print genname_counts

{('Action', 'Animation'): 199, ('Western', 'Western'): 1042, ('Adventure', 'War'): 117, ('Action', 'Crime'): 1193, ('Action', 'Science-Fiction'): 890, ('Foreign', 'War'): 11, ('War', 'Science-Fiction'): 3, ('Thriller', 'Action'): 279, ('Action', 'Music'): 31, ('Mystery', 'TV-Movie'): 21, ('Romance', 'Family'): 99, ('Western', 'Mystery'): 4, ('Foreign', 'Mystery'): 26, ('Drama', 'TV-Movie'): 171, ('Science-Fiction', 'War'): 22, ('Western', 'Foreign'): 6, ('Adventure', 'Romance'): 263, ('Fantasy', 'Horror'): 281, ('Mystery', 'Family'): 13, ('Fantasy', 'Action'): 187, ('Horror', 'Comedy'): 215, ('Animation', 'Science-Fiction'): 240, ('TV-Movie', 'Crime'): 44, ('History', 'TV-Movie'): 14, ('Animation', 'Crime'): 22, ('War', 'History'): 112, ('Music', 'Mystery'): 13, ('Science-Fiction', 'Foreign'): 50, ('Fantasy', 'War'): 12, ('Fantasy', 'Science-Fiction'): 428, ('Foreign', 'Crime'): 13, ('Drama', 'Fantasy'): 444, ('Crime', 'TV-Movie'): 23, ('Mystery', 'Fantasy'): 41, ('Horror', 'Western'):

In [81]:
import operator
for w in sorted(gen_counts, key=gen_counts.get, reverse=True):
    val = gen_counts[w]
    g1 = id_to_genre[w[0]]
    g2 = id_to_genre[w[1]]
    print(g1, g2, val)

('Drama', 'Drama', 20265)
('Comedy', 'Comedy', 13182)
('Thriller', 'Thriller', 7624)
('Romance', 'Romance', 6735)
('Action', 'Action', 6596)
('Horror', 'Horror', 4673)
('Crime', 'Crime', 4307)
('Documentary', 'Documentary', 3932)
('Drama', 'Romance', 3682)
('Adventure', 'Adventure', 3496)
('Science-Fiction', 'Science-Fiction', 3049)
('Comedy', 'Drama', 2858)
('Drama', 'Thriller', 2780)
('Family', 'Family', 2770)
('Comedy', 'Romance', 2580)
('Mystery', 'Mystery', 2467)
('Fantasy', 'Fantasy', 2313)
('Action', 'Thriller', 2091)
('Animation', 'Animation', 1935)
('Crime', 'Drama', 1699)
('Foreign', 'Foreign', 1622)
('Action', 'Drama', 1599)
('Music', 'Music', 1598)
('Horror', 'Thriller', 1524)
('History', 'History', 1398)
('Drama', 'Comedy', 1333)
('War', 'War', 1323)
('Action', 'Adventure', 1307)
('Crime', 'Thriller', 1266)
('Action', 'Crime', 1193)
('Action', 'Comedy', 1184)
('Western', 'Western', 1042)
('Mystery', 'Thriller', 975)
('Drama', 'Mystery', 962)
('Comedy', 'Family', 957)
('Dra

In [84]:
diffdict = {}
for k,v in gen_counts.items():
    if k[0] != k[1]:
        p1 = gen_counts[k] * 1.0 / gen_counts[(k[0],k[0])]
        p2 = gen_counts[k] * 1.0 /gen_counts[(k[1],k[1])]
        diff = p1 - p2
        diffdict[k] = diff

        

In [85]:
# getting a crude map

for w in sorted(diffdict, key=diffdict.get, reverse=True):
    val = diffdict[w]
    g1 = id_to_genre[w[0]]
    g2 = id_to_genre[w[1]]
    print(g1, g2, val)

('Crime', 'Drama', 0.3106349804033175)
('TV-Movie', 'Drama', 0.2722123519172786)
('Mystery', 'Thriller', 0.26733123824196076)
('Adventure', 'Drama', 0.20876501345727463)
('War', 'Drama', 0.20630142673073837)
('Animation', 'Comedy', 0.2059165716933709)
('Music', 'Romance', 0.179466352242456)
('Action', 'Drama', 0.16351513309773458)
('Adventure', 'Comedy', 0.13766800959772132)
('History', 'Drama', 0.13519016804168799)
('Crime', 'Thriller', 0.12788553298261868)
('Horror', 'Thriller', 0.12623375696018002)
('Animation', 'Family', 0.11029580499818092)
('TV-Movie', 'Comedy', 0.1056011150284543)
('Science-Fiction', 'Thriller', 0.1011611843905266)
('Fantasy', 'Comedy', 0.09803143106161548)
('Fantasy', 'Drama', 0.09153527690653958)
('Action', 'Comedy', 0.08968327816042587)
('TV-Movie', 'Romance', 0.08895832062945422)
('Animation', 'Drama', 0.08694564462729301)
('Romance', 'Drama', 0.07990034053248796)
('Music', 'Drama', 0.07897174082950344)
('History', 'Romance', 0.0776556391408662)
('Mystery', 

('Thriller', 'TV-Movie', -0.05394034620651726)
('Adventure', 'War', -0.0549685549276919)
('Action', 'Mystery', -0.05658482140113661)
('Documentary', 'Foreign', -0.05686529196086868)
('Comedy', 'War', -0.05779973446921883)
('Romance', 'Western', -0.05840746551304028)
('Comedy', 'Horror', -0.05843079599722634)
('Documentary', 'History', -0.05900596259443443)
('Thriller', 'Science-Fiction', -0.06120842090555209)
('Adventure', 'History', -0.06138509410305012)
('Adventure', 'Animation', -0.06299602646657089)
('Romance', 'TV-Movie', -0.06931817191905525)
('Romance', 'War', -0.07045605671122693)
('Family', 'TV-Movie', -0.07259330035442133)
('Action', 'Animation', -0.07267257738226754)
('Drama', 'Action', -0.07403687077095675)
('Thriller', 'Crime', -0.07576157167216745)
('Comedy', 'Mystery', -0.0757826093079845)
('Adventure', 'Western', -0.07814359816757951)
('Adventure', 'Fantasy', -0.08061006785841837)
('Documentary', 'Music', -0.08209237895283653)
('Horror', 'Science-Fiction', -0.0828642550